In [1]:
class BinaryTree:
    def __init__(self, key, leftTree=None, rightTree=None):
        self.key = key
        self.leftTree = leftTree
        self.rightTree = rightTree

    def setKey(self, key):
        self.key = key

    def getKey(self):
        return self.key

    def getLeftTree(self):
        return self.leftTree

    def getRightTree(self):
        return self.rightTree

    def insertLeft(self, key):
        if self.leftTree is None:
            self.leftTree = BinaryTree(key)
        else:
            new_node = BinaryTree(key, leftTree=self.leftTree)
            self.leftTree = new_node

    def insertRight(self, key):
        if self.rightTree is None:
            self.rightTree = BinaryTree(key)
        else:
            new_node = BinaryTree(key, rightTree=self.rightTree)
            self.rightTree = new_node

    def printPreorder(self, level=0):
        print(str(level * '-') + str(self.key))
        if self.leftTree is not None:
            self.leftTree.printPreorder(level + 1)
        if self.rightTree is not None:
            self.rightTree.printPreorder(level + 1)
            
    def printInorder(self, level=0):
        if self.leftTree is not None:
            self.leftTree.printInorder(level + 1)
        print(str(level * '-') + str(self.key))
        if self.rightTree is not None:
            self.rightTree.printInorder(level + 1)

    def printPostorder(self, level=0):
        if self.leftTree is not None:
            self.leftTree.printPostorder(level + 1)
        if self.rightTree is not None:
            self.rightTree.printPostorder(level + 1)
        print(str(level * '-') + str(self.key))


class Stack:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        if not self.isEmpty():
            return self.items.pop()
        else:
            raise IndexError("Pop from an empty stack")

    def peek(self):
        if not self.isEmpty():
            return self.items[-1]
        else:
            raise IndexError("Peek from an empty stack")

    def size(self):
        return len(self.items)


class Node:
    def __init__(self, key, item):
        self.key = key
        self.item = item
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def append(self, key, item):
        new_node = Node(key, item)
        if self.head is None:
            self.head = new_node
        else:
            current = self.head
            while current.next:
                current = current.next
            current.next = new_node

    def find(self, key):
        current = self.head
        while current:
            if current.key == key:
                return current.item
            current = current.next
        return None

    def sort(self):
        if self.head is None or self.head.next is None:
            return

        size = 1
        current = self.head
        while current.next:
            size += 1
            current = current.next

        step = 1
        while step < size:
            self.head = self.merge_pass(self.head, step)
            step *= 2

    def merge_pass(self, head, step):
        current = head
        prev_tail = None
        new_head = None

        while current:
            left = current
            right = self.split(left, step)
            current = self.split(right, step)

            merged = self.sorted_merge(left, right)

            if prev_tail:
                prev_tail.next = merged
            else:
                new_head = merged

            while prev_tail and prev_tail.next:
                prev_tail = prev_tail.next

        return new_head

    def split(self, head, step):
        if head is None:
            return None
        for _ in range(step - 1):
            if head.next is None:
                break
            head = head.next
        next_head = head.next
        head.next = None
        return next_head

    def sorted_merge(self, a, b):
        if a is None:
            return b
        if b is None:
            return a

        if a.key <= b.key:
            result = a
            result.next = self.sorted_merge(a.next, b)
        else:
            result = b
            result.next = self.sorted_merge(a, b.next)
        return result

In [2]:

def isvariable(char):
    # Define criteria for what you consider a variable
    return char.isalpha()

def isoperator(char):
    # Define criteria for what you consider an operator
    return char in {'+', '-', '*', '/', '^'}

def tokenize_inner(exp):
        tokens = []
        i = 0
        while i < len(exp):
            if exp[i] == '*' and i + 1 < len(exp) and exp[i + 1] == '*':
                tokens.append('**')
                i += 2
            elif exp[i] == '-':
                if (i != 0 and (exp[i - 1]).isdigit() or isvariable(exp[i - 1]) or exp[i - 1] == ')') and ((exp[i + 1]).isdigit() or isvariable(exp[i + 1]) or isoperator(exp[i + 1])):
                    tokens.append(exp[i])
                    i += 1
                else:
                    if (exp[i + 1]).isdigit() or isvariable(exp[i + 1]):
                        tokens.extend(['(', exp[i + 1], '*', '-1', ')'])
                        i += 2
                    elif exp[i + 1] == '(':
                        # Find the matching closing bracket
                        j = i + 1
                        bracket_count = 1
                        while j < len(exp) and bracket_count > 0:
                            j += 1
                            if exp[j] == '(':
                                bracket_count += 1
                            elif exp[j] == ')':
                                bracket_count -= 1
                        # Recursive call for expression inside the brackets
                        inside_tokens = tokenize_inner(exp[i + 2:j])
                        tokens.extend(['(', *inside_tokens, ')', '*', '-1', ')'])
                        i = j + 1
                    else:
                        print("Error")
                        break
            else:
                tokens.append(exp[i])
                i += 1
        return tokens

def tokenize(expression):
    exp = expression.replace(" ", "")
    return tokenize_inner(exp)


In [3]:
expression = "a = 5**-(1 -2)"
tokenize(expression)

['a', '=', '5', '**', '(', '1', '-', '2', ')', '*', '-1', ')']

In [4]:
import re
from collections import deque

class Token:
    def __init__(self, type_, value):
        self.type = type_
        self.value = value

class ParseTreeNode:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

def tokenize(expression):
    token_specification = [
        ("NUMBER",   r"\d+(\.\d*)?"),
        ("OPERATOR", r"[+\-*/**]"),
        ("VARIABLE", r"[a-zA-Z]+"),
        ("LPAREN",   r"\("),
        ("RPAREN",   r"\)"),
    ]
    token_regex = '|'.join('(?P<%s>%s)' % pair for pair in token_specification)
    for mo in re.finditer(token_regex, expression):
        kind = mo.lastgroup
        value = mo.group()
        if kind == "NUMBER":
            value = float(value) if '.' in value else int(value)
        yield Token(kind, value)

def build_parse_tree(tokens):
    root = None
    current_node = None
    stack = deque()

    for token in tokens:
        if token.type in ["NUMBER", "VARIABLE"]:
            new_node = ParseTreeNode(token)
            if current_node:
                if current_node.left is None:
                    current_node.left = new_node
                else:
                    current_node.right = new_node
            else:
                root = new_node
        elif token.type == "OPERATOR":
            new_node = ParseTreeNode(token)
            if root is not None:
                new_node.left = root
            root = new_node
            current_node = new_node
        elif token.type == "LPAREN":
            stack.append((root, current_node))
            root = None
            current_node = None
        elif token.type == "RPAREN":
            if stack:
                prev_root, prev_current_node = stack.pop()
                if prev_current_node:
                    if prev_current_node.left is None:
                        prev_current_node.left = root
                    else:
                        prev_current_node.right = root
                else:
                    prev_root = root
                root = prev_root
                current_node = prev_current_node

    return root if root else current_node

def evaluate_parse_tree(node, variable_values):
    if node.value.type == "NUMBER":
        return node.value.value
    elif node.value.type == "VARIABLE":
        return variable_values.get(node.value.value, None)
    elif node.value.type == "OPERATOR":
        left_val = evaluate_parse_tree(node.left, variable_values) if node.left else None
        right_val = evaluate_parse_tree(node.right, variable_values) if node.right else None

        if left_val is not None and right_val is not None:
            if node.value.value == '+':
                return left_val + right_val
            elif node.value.value == '-':
                return left_val - right_val
            elif node.value.value == '*':
                return left_val * right_val
            elif node.value.value == '/':
                return left_val / right_val if right_val != 0 else None
            elif node.value.value == '**':
                return left_val ** right_val
        else:
            return None

    return None

def parse_equation(equation):
    variable, expression = equation.split('=')
    variable = variable.strip()
    expression = expression.strip()
    return variable, expression

def solve_equations(equations):
    variable_values = {}
    for equation in equations:
        variable, expression = parse_equation(equation)
        tokens = tokenize(expression)
        parse_tree = build_parse_tree(tokens)
        if parse_tree:
            result = evaluate_parse_tree(parse_tree, variable_values)
            variable_values[variable] = result

    # Assign 'None' to variables that couldn't be resolved
    for variable in variable_values:
        if variable_values[variable] is None:
            variable_values[variable] = 'None'

    return variable_values

# Example usage with a set of equations
equations = [
    "c = a + b",
    "b = a ** 2",
    "a = (3 + (4 * 5))",
    "d = x"
]

# Solve the equations
results = solve_equations(equations)

# Format and display the results
formatted_results = [f"{var} = {res}" for var, res in results.items()]
print('\n'.join(formatted_results))


c = None
b = None
a = 23
d = None


In [5]:
equations = [
    "d = x"
    "b = a + c",
    "c = a ** 2",
    "a = (3 + (4 * 5))",
]

In [6]:
equations = {
    "a": "(3 + (4 * 5))",
    "c": "a ** 2",
    "b": "a + c",
    "d": "x"
}

In [8]:
def sort_equations(equations):
    # Parsing the equations to extract variables and their dependencies
    eq_dict = {}
    all_deps = set()
    for eq in equations:
        var, expr = eq.split(" = ")
        deps = set([x for x in expr.split(" ") if x.isalpha() and x != var])
        eq_dict[var] = {"expr": expr, "deps": deps}
        all_deps.update(deps)

    # Identifying undefined variables
    undefined_vars = all_deps - eq_dict.keys()

    # Sorting the equations based on dependencies
    sorted_eqs = sorted(eq_dict.items(), key=lambda x: (len(x[1]["deps"]), x[0]))

    # Preparing the final sorted dictionary
    sorted_dict = {var: details["expr"] for var, details in sorted_eqs}

    # Adding undefined variables at the end
    for var in undefined_vars:
        sorted_dict[var] = "undefined"

    return sorted_dict

# Test with the given equations
equations = [
    "d = x",
    "b = a + c",
    "a = (3 + (4 * 5))",
    "c = a ** 2"
]

sorted_equations = sort_equations(equations)
sorted_equations


{'a': '(3 + (4 * 5))', 'c': 'a ** 2', 'd': 'x', 'b': 'a + c', 'x': 'undefined'}